In [1]:
import flair
import csv
from flair.models import SequenceTagger
from flair.data import Sentence
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings, FlairEmbeddings, CharacterEmbeddings, OneHotEmbeddings
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import os
import json

In [2]:
#Load Model
model_path = "Model/best-model.pt"
model = SequenceTagger.load(model_path)

2020-10-22 12:40:42,758 loading file Model/best-model.pt


In [3]:
#Load Testdata
testdata_path = "Data/testdata.json"

In [4]:
# Tagging Beispiel
sentence = Sentence('we will rock you')
model.predict(sentence)
print(sentence.to_tagged_string())

we <B_C> will rock you


In [5]:
with open(testdata_path,'r', encoding='utf8') as infile:
    example_data = json.load(infile)


for i,(k,v) in enumerate(example_data.items()):
    tokens = v.get('tokens')
    tokens = [token.lower() for token in tokens]
    example_data[k]['tokens'] = tokens
    
all_tokens = [v['tokens'] for k,v in example_data.items()]
all_labels = [v['labels'] for k,v in example_data.items()]

In [7]:
if os.path.exists("Data/Modified/modified_test.txt"):
  os.remove("Data/Modified/modified_test.txt")
else:
  print("The file does not exist")

f = open("Data/Modified/modified_test.txt", "x")

for x in range(0, len(all_tokens)):
    for y in range(0, len(all_tokens[x])):
        new_line = all_tokens[x][y] + " " + all_labels[x][y] + "\n"
        f.write(new_line)
    f.write("\n")
    
f.close()

In [8]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = './'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='Data/Modified/dummy.txt',
                              test_file='Data/Modified/modified_test.txt',
                              dev_file='Data/Modified/dummy.txt')

2020-10-22 12:42:16,265 Reading data from .
2020-10-22 12:42:16,267 Train: Data/Modified/dummy.txt
2020-10-22 12:42:16,268 Dev: Data/Modified/dummy.txt
2020-10-22 12:42:16,270 Test: Data/Modified/modified_test.txt


In [9]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings, FlairEmbeddings, CharacterEmbeddings, OneHotEmbeddings
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)


Dictionary with 20 tags: <unk>, O, B_C, B_E, B_P, I_P, I_E, B_M, I_M, B_A, B_T, I_T, B_V, I_V, B_W, I_C, I_A, I_W, <START>, <STOP>


In [36]:
embedding_types = [

    #WordEmbeddings('glove'),
    OneHotEmbeddings(corpus),
    # TransformerWordEmbeddings("bert-base-uncased"),
    # comment in this line to use character embeddings
    #CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    #FlairEmbeddings('news-forward'),
    #FlairEmbeddings('news-backward'),
    #ELMoEmbeddings('medium')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

[b'<unk>']
vocabulary size of 1


In [37]:
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256, #256, 512
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        #loss_weights = weight_dict,
                                        rnn_layers = 2, #Number of rnn_layers
                                        tag_type=tag_type,
                                        use_crf=True)


In [38]:
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

In [39]:
# Dummy Training für Testdatenprognose
trainer.train('Model',
              learning_rate=0.0,
              mini_batch_size=32,
              max_epochs=0,
              train_with_dev = "False", #Vorsicht-> eigentlich False
             embeddings_storage_mode = "cpu")

2020-10-22 12:20:31,161 ----------------------------------------------------------------------------------------------------
2020-10-22 12:20:31,163 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): OneHotEmbeddings(
      min_freq=3
      (embedding_layer): Embedding(1, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (linear): Linear(in_features=512, out_features=15, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-10-22 12:20:31,166 ----------------------------------------------------------------------------------------------------
2020-10-22 12:20:31,167 Corpus: "Corpus: 1 train + 1 dev + 26 test sentences"
2020-10-22 12:20:31,169 --------------------------------------------------------------------------------------------

{'test_score': 0.9602,
 'dev_score_history': [],
 'train_loss_history': [],
 'dev_loss_history': []}

In [40]:
test_results= "Model/test.tsv"
tsv_file = open(test_results)
tsv_data = csv.reader(tsv_file, delimiter =" ")
test_data = []
end = ['END', 'END', 'END']

for row in tsv_data:
    if not row:
        test_data.append(end)
    else:
        test_data.append(row)

tsv_file.close()

In [41]:
tokens =[]
labels =[]
predicted = []
predicted_list = []
label_list = []
token_list = []

for token in test_data:
    if token == end:
        token_list.append(tokens)
        label_list.append(labels)
        predicted_list.append(predicted)
        tokens =[]
        labels =[]
        predicted = []
    else:
        tokens.append(token[0])
        labels.append(token[1])
        predicted.append(token[2])

In [42]:
test_tokens = token_list
test_labels = label_list


In [43]:
labelclass_to_id={'O':0,
'B_C':1,'I_C':2, # 'companionship'
'B_T':3,'I_T':4, # 'time'
'B_W':5,'I_W':6, # 'weather'
'B_P':7,'I_P':8, # 'position'
'B_M':9,'I_M':10, # 'motivation'
'B_E':11,'I_E':12, # 'emotional_state'
'B_A':13,'I_A':14, # 'activity'
'B_V':15,'I_V':16# 'visited_before'
}

max_seq_length = 120
n_tags = len(list(labelclass_to_id.keys()))


In [49]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

predicted_list_flat = [item for sublist in predicted_list for item in sublist]
label_list_flat = [item for sublist in label_list for item in sublist]
y_pred = predicted_list_flat
y_true = label_list_flat


labels = list(set(y_true))
labels.sort()

print("Anzahl der Labels: %s -> %s" % (len(labels), labels))

Anzahl der Labels: 12 -> ['B_A', 'B_C', 'B_E', 'B_P', 'B_T', 'B_W', 'I_A', 'I_E', 'I_P', 'I_T', 'I_W', 'O']


In [45]:
df = pd.DataFrame(
    data=confusion_matrix(y_true, y_pred, labels=labels),
    columns=labels,
    index=labels
)
df

,B_A,B_C,B_E,B_P,B_T,B_W,I_A,I_E,I_P,I_T,I_W,O
B_A,0,0,0,0,0,0,0,0,0,0,0,1
B_C,0,2,0,0,0,0,0,0,0,0,0,2
B_E,0,0,2,0,0,0,0,0,0,0,0,1
B_P,0,0,0,0,0,0,0,0,0,0,0,1
B_T,0,0,0,0,0,0,0,0,0,0,0,1
B_W,0,0,0,0,0,0,0,0,0,0,0,1
I_A,0,0,0,0,0,0,0,0,0,0,0,1
I_E,0,0,1,0,0,0,0,0,0,0,0,0
I_P,0,0,0,0,0,0,0,0,0,0,0,1
I_T,0,0,0,0,0,0,0,0,0,0,0,1


In [46]:
conf_matrix = df

In [47]:
# compute final evaluation measures
precision_per_class = np.zeros((n_tags,))
recall_per_class = np.zeros((n_tags,))
for i in range(n_tags):
    if conf_matrix.values[i,i] > 0:
        precision_per_class[i] = conf_matrix.values[i,i]/sum(conf_matrix.values[:,i])
        recall_per_class[i] = conf_matrix.values[i,i]/sum(conf_matrix.values[i,:])
precision = np.mean(precision_per_class)
recall = np.mean(recall_per_class)
f1 = 2*(precision*recall)/(precision+recall)

print('Precision: '+str(precision))
print('Recall: '+str(recall))
print('F1-measure: '+str(f1))

IndexError: index 12 is out of bounds for axis 0 with size 12